# LangChain: Models, Prompts and Output Parsers


## Outline

 * Direct API calls
 * API calls through LangChain:
   * Prompts
   * Models
   * Output parsers

## Get your [OpenAI API Key](https://platform.openai.com/account/api-keys)

In [101]:
!pip install -q -U langchain langchain-openai  openai langchain_community

In [102]:
from google.colab import userdata
api_key = userdata.get('API_KEY')

Note: LLM's do not always produce the same results. When executing the code in your notebook, you may get slightly different answers that those in the video.

## Chat API : OpenAI compatiable with other models

Let's start with a direct API calls.

In [147]:
from openai import OpenAI

# gets API Key from environment variable
client = OpenAI(
  base_url= "https://openrouter.ai/api/v1",
  api_key= api_key,
)

In [148]:
def get_completion(prompt, model='microsoft/phi-3-medium-128k-instruct:free'):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0.2,
    )
    return response.choices[0].message.content


In [149]:
output = get_completion("What is 1+1?")
output

' 1 + 1 equals 2.'

In [106]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [107]:
style = """American English \
in a calm and respectful tone
"""

In [108]:
prompt = f"""Translate the text \
that is delimited by triple backticks
into a style that is {style}.
text: ```{customer_email}```
"""

print(prompt)

Translate the text that is delimited by triple backticks
into a style that is American English in a calm and respectful tone
.
text: ```
Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse,the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!
```



In [109]:
response = get_completion(prompt, model ='microsoft/phi-3-medium-128k-instruct:free')

In [110]:
response

" Well, I'm quite upset that my blender lid flew off and splattered my kitchen walls with smoothie! To make matters worse, the warranty doesn't cover the cost of cleaning up my kitchen. I need your help right now, friend!"

## Chat API : LangChain

Let's try how we can do the same using LangChain.

### Model

In [111]:
from langchain.chat_models import ChatOpenAI

In [112]:
# To control the randomness and creativity of the generated
# text by an LLM, use temperature = 0.0
chat = ChatOpenAI(model='microsoft/phi-3-medium-128k-instruct:free' , temperature=0.2, api_key=api_key, base_url="https://openrouter.ai/api/v1")
chat

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x77fe44be5510>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x77fe44a662f0>, model_name='microsoft/phi-3-medium-128k-instruct:free', temperature=0.2, openai_api_key='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', openai_api_base='https://openrouter.ai/api/v1', openai_proxy='')

### Prompt template

In [113]:
template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

In [114]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(template_string)


In [115]:
prompt_template.messages[0].prompt

PromptTemplate(input_variables=['style', 'text'], template='Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```\n')

In [116]:
prompt_template.messages[0].prompt.input_variables

['style', 'text']

In [117]:
customer_style = """American English \
in a calm and respectful tone
"""

In [118]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [119]:
customer_messages = prompt_template.format_messages(
                    style=customer_style,
                    text=customer_email)

In [120]:
print(type(customer_messages))
print(type(customer_messages[0]))

<class 'list'>
<class 'langchain_core.messages.human.HumanMessage'>


In [121]:
print(customer_messages[0])

content="Translate the text that is delimited by triple backticks into a style that is American English in a calm and respectful tone\n. text: ```\nArrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!\n```\n"


In [122]:
# Call the LLM to translate to the style of the customer message
customer_response = chat(customer_messages)

In [123]:
print(customer_response.content)

 Well, I'm quite upset that my blender lid flew off and splattered my kitchen walls with smoothie! To make matters worse, the warranty doesn't cover the cost of cleaning up my kitchen. I need your help right now, friend!


In [124]:
service_reply = """Hey there customer, \
the warranty does not cover \
cleaning expenses for your kitchen \
because it's your fault that \
you misused your blender \
by forgetting to put the lid on before \
starting the blender. \
Tough luck! See ya!
"""

In [125]:
service_style_formal = """\
a polite tone \
that speaks in English Formal\
"""

In [126]:
service_messages = prompt_template.format_messages(
    style=service_style_formal,
    text=service_reply)

print(service_messages[0].content)

Translate the text that is delimited by triple backticks into a style that is a polite tone that speaks in English Formal. text: ```Hey there customer, the warranty does not cover cleaning expenses for your kitchen because it's your fault that you misused your blender by forgetting to put the lid on before starting the blender. Tough luck! See ya!
```



In [127]:
service_response = chat(service_messages)
print(service_response.content)

 Dear valued customer,

I regret to inform you that the warranty does not cover the cleaning expenses for your kitchen, as the issue arose due to the blender being used without the lid. We understand that this may be disappointing news, and we appreciate your understanding in this matter.

Thank you for your continued support.

Best regards,
[Your Name]


## Output Parsers

Let's start with defining how we would like the LLM output to look like:

In [128]:
{
  "gift": False,
  "delivery_days": 5,
  "price_value": "pretty affordable!"
}

{'gift': False, 'delivery_days': 5, 'price_value': 'pretty affordable!'}

In [129]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [130]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

input_variables=['text'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], template='For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\nFormat the output as JSON with the following keys:\ngift\ndelivery_days\nprice_value\n\ntext: {text}\n'))]


In [131]:
messages = prompt_template.format_messages(text=customer_review)
# chat = ChatOpenAI(temperature=0.0, model=llm_model)
chat = ChatOpenAI(model='microsoft/phi-3-medium-128k-instruct:free' , temperature=0, api_key=api_key, base_url="https://openrouter.ai/api/v1")
response = chat(messages)
print(response.content)

 {
  "gift": True,
  "delivery_days": 2,
  "price_value": ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]
}


In [132]:
type(response.content)

str

In [133]:
# You will get an error by running this line of code
# because'gift' is not a dictionary
# 'gift' is a string
response.content.get('gift')

AttributeError: 'str' object has no attribute 'get'

### Parse the LLM output string into a Python dictionary

In [134]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [135]:
gift_schema = ResponseSchema(name="gift",
                             description="Was the item purchased\
                             as a gift for someone else? \
                             Answer True if yes,\
                             False if not or unknown.")
delivery_days_schema = ResponseSchema(name="delivery_days",
                                      description="How many days\
                                      did it take for the product\
                                      to arrive? If this \
                                      information is not found,\
                                      output -1.")
price_value_schema = ResponseSchema(name="price_value",
                                    description="Extract any\
                                    sentences about the value or \
                                    price, and output them as a \
                                    comma separated Python list.")

response_schemas = [gift_schema,
                    delivery_days_schema,
                    price_value_schema]

In [136]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [137]:
format_instructions = output_parser.get_format_instructions()

In [138]:
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"gift": string  // Was the item purchased                             as a gift for someone else?                              Answer True if yes,                             False if not or unknown.
	"delivery_days": string  // How many days                                      did it take for the product                                      to arrive? If this                                       information is not found,                                      output -1.
	"price_value": string  // Extract any                                    sentences about the value or                                     price, and output them as a                                     comma separated Python list.
}
```


In [139]:
review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=review_template_2)

messages = prompt.format_messages(text=customer_review,
                                format_instructions=format_instructions)

In [140]:
print(messages[0].content)

For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the productto arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,and output them as a comma separated Python list.

text: This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```

In [141]:
response = chat(messages)

In [142]:
print(response.content)

 ```json
{
	"gift": "True",
	"delivery_days": "2",
	"price_value": "It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."
}
```


In [143]:
output_dict = output_parser.parse(response.content)

In [144]:
output_dict

{'gift': 'True',
 'delivery_days': '2',
 'price_value': "It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."}

In [145]:
type(output_dict)

dict

In [146]:
output_dict.get('delivery_days')

'2'